# Create SRA submission sheet
This Python Jupyter notebook creates a BioSample submission sheet for the Sequence Read Archive:

First, import Python modules:

In [1]:
import datetime
import glob
import itertools
import os

import natsort

import pandas as pd

import yaml

Display full columns of data frames:

In [2]:
pd.set_option('display.max_colwidth', None)

Read the configuration for the uploads:

In [3]:
with open('upload_config.yaml') as f:
    config = yaml.safe_load(f)

Read the Illumina runs to submit:

In [4]:
print(f"Reading Illumina runs from {config['barcode_runs']}")

illumina_runs = pd.read_csv(config['barcode_runs'])

print('Here are the first few entries in the Illumina runs:')
illumina_runs.head()

Reading Illumina runs from barcode_runs_RBD-nano-vax-NHP1.csv
Here are the first few entries in the Illumina runs:


library                          sample    date    experiment   antibody  \
0    lib1   expt_107-116-none-0-reference  210216  expt_107-116       none   
1    lib2   expt_107-116-none-0-reference  210216  expt_107-116       none   
2    lib1  expt_115-NHP-serum-1000-escape  210216      expt_115  NHP-serum   
3    lib2  expt_115-NHP-serum-1000-escape  210216      expt_115  NHP-serum   

   concentration concentration_units         group  selection  frac_escape  \
0              0            dilution       Moderna  reference          NaN   
1              0            dilution       Moderna  reference          NaN   
2           1000            dilution  Lexi_vaccine     escape        0.047   
3           1000            dilution  Lexi_vaccine     escape        0.051   

   cells_sorted  \
0           NaN   
1           NaN   
2      454104.0   
3      457684.0   

                                                                                                                          R1  
0  /shared/ngs/illumina/agreaney/210219_D00300_1182_AHKH22BCX3/Unaligned/Project_agreaney/expt_97-106_lib1_ref_*R1*.fastq.gz  
1  /shared/ngs/illumina/agreaney/210219_D00300_1182_AHKH22BCX3/Unaligned/Project_agreaney/expt_97-106_lib2_ref_*R1*.fastq.gz  
2   /shared/ngs/illumina/agreaney/210219_D00300_1182_AHKH22BCX3/Unaligned/Project_agreaney/expt_115_lib1_abneg_*R1*.fastq.gz  
3   /shared/ngs/illumina/agreaney/210219_D00300_1182_AHKH22BCX3/Unaligned/Project_agreaney/expt_115_lib2_abneg_*R1*.fastq.gz

Next make submission entries for the barcode runs, initially in "tidy" format with one FASTQ file per row:

In [5]:
submissions_tidy = (
    illumina_runs
    .assign(
        biosample_accession=lambda x: config['biosample_accession'],
        library_ID=lambda x: x['library'] + '_' + x['sample'],
        title=lambda x: 'SARS-CoV-2 RBD mutational antig Illumina barcode sequencing for ' + x['sample'],
        library_strategy='AMPLICON',
        library_source='SYNTHETIC',
        library_selection='PCR',
        library_layout='single',
        platform='ILLUMINA',
        instrument_model='Illumina HiSeq 2500',
        design_description='PCR of barcodes from RBD variants',
        filetype='fastq',
        filename_fullpath=lambda x: x['R1'].str.split(';')
                                    .map(lambda flist: list(itertools.chain.from_iterable(glob.glob(f) for f in flist))),       
        )
    .explode('filename_fullpath')
    .assign(filename_fullpath=lambda x: x['filename_fullpath'].str.strip(),
            filename=lambda x: x['filename_fullpath'].map(os.path.basename))
    .drop(columns=illumina_runs.columns)
    .reset_index(drop=True)
    )

assert submissions_tidy['filename_fullpath'].map(os.path.isfile).all()

print('Here are the first few submission entries in tidy format:')
submissions_tidy.head()

Here are the first few submission entries in tidy format:


biosample_accession                           library_ID  \
0        SAMN18322429   lib1_expt_107-116-none-0-reference   
1        SAMN18322429   lib2_expt_107-116-none-0-reference   
2        SAMN18322429  lib1_expt_115-NHP-serum-1000-escape   
3        SAMN18322429  lib2_expt_115-NHP-serum-1000-escape   

                                                                                            title  \
0   SARS-CoV-2 RBD mutational antig Illumina barcode sequencing for expt_107-116-none-0-reference   
1   SARS-CoV-2 RBD mutational antig Illumina barcode sequencing for expt_107-116-none-0-reference   
2  SARS-CoV-2 RBD mutational antig Illumina barcode sequencing for expt_115-NHP-serum-1000-escape   
3  SARS-CoV-2 RBD mutational antig Illumina barcode sequencing for expt_115-NHP-serum-1000-escape   

  library_strategy library_source library_selection library_layout  platform  \
0         AMPLICON      SYNTHETIC               PCR         single  ILLUMINA   
1         AMPLICON      SYNTHETIC               PCR         single  ILLUMINA   
2         AMPLICON      SYNTHETIC               PCR         single  ILLUMINA   
3         AMPLICON      SYNTHETIC               PCR         single  ILLUMINA   

      instrument_model                 design_description filetype  \
0  Illumina HiSeq 2500  PCR of barcodes from RBD variants    fastq   
1  Illumina HiSeq 2500  PCR of barcodes from RBD variants    fastq   
2  Illumina HiSeq 2500  PCR of barcodes from RBD variants    fastq   
3  Illumina HiSeq 2500  PCR of barcodes from RBD variants    fastq   

                                                                                                                 filename_fullpath  \
0  /shared/ngs/illumina/agreaney/210219_D00300_1182_AHKH22BCX3/Unaligned/Project_agreaney/expt_97-106_lib1_ref_S21_R1_001.fastq.gz   
1  /shared/ngs/illumina/agreaney/210219_D00300_1182_AHKH22BCX3/Unaligned/Project_agreaney/expt_97-106_lib2_ref_S22_R1_001.fastq.gz   
2   /shared/ngs/illumina/agreaney/210219_D00300_1182_AHKH22BCX3/Unaligned/Project_agreaney/expt_115_lib1_abneg_S17_R1_001.fastq.gz   
3   /shared/ngs/illumina/agreaney/210219_D00300_1182_AHKH22BCX3/Unaligned/Project_agreaney/expt_115_lib2_abneg_S18_R1_001.fastq.gz   

                                   filename  
0  expt_97-106_lib1_ref_S21_R1_001.fastq.gz  
1  expt_97-106_lib2_ref_S22_R1_001.fastq.gz  
2   expt_115_lib1_abneg_S17_R1_001.fastq.gz  
3   expt_115_lib2_abneg_S18_R1_001.fastq.gz

For the actual submission, we need a "wide" data frame that for each unique `sample_name` / `library_ID` gives all of the files each in different columns.
These should be files without the full path.

First, look at how many files there are for each sample / library:

In [6]:
(submissions_tidy
 .groupby(['biosample_accession', 'library_ID'])
 .aggregate(n_files=pd.NamedAgg('filename_fullpath', 'count'))
 .sort_values('n_files', ascending=False)
 .reset_index()
 )

biosample_accession                           library_ID  n_files
0        SAMN18322429   lib1_expt_107-116-none-0-reference        1
1        SAMN18322429  lib1_expt_115-NHP-serum-1000-escape        1
2        SAMN18322429   lib2_expt_107-116-none-0-reference        1
3        SAMN18322429  lib2_expt_115-NHP-serum-1000-escape        1

Now make the wide submission data frame.
Note we keep only the filename column with the path lacking the full directory information:

In [7]:
submissions_wide = (
    submissions_tidy
    .assign(
        filename_count=lambda x: x.groupby(['biosample_accession', 'library_ID'])['filename'].cumcount() + 1,
        filename_col=lambda x: 'filename' + x['filename_count'].map(lambda c: str(c) if c > 1 else '')
        )
    .pivot(
        index='library_ID',
        columns='filename_col',
        values='filename',
        )
    )

submissions_wide = (
    submissions_tidy
    .drop(columns=['filename_fullpath', 'filename'])
    .drop_duplicates()
    .merge(submissions_wide[natsort.natsorted(submissions_wide.columns)],
           on='library_ID',
           validate='one_to_one',
           )
    )

print('Here are the first few submission entries in wide format:')
submissions_wide.head()

Here are the first few submission entries in wide format:


biosample_accession                           library_ID  \
0        SAMN18322429   lib1_expt_107-116-none-0-reference   
1        SAMN18322429   lib2_expt_107-116-none-0-reference   
2        SAMN18322429  lib1_expt_115-NHP-serum-1000-escape   
3        SAMN18322429  lib2_expt_115-NHP-serum-1000-escape   

                                                                                            title  \
0   SARS-CoV-2 RBD mutational antig Illumina barcode sequencing for expt_107-116-none-0-reference   
1   SARS-CoV-2 RBD mutational antig Illumina barcode sequencing for expt_107-116-none-0-reference   
2  SARS-CoV-2 RBD mutational antig Illumina barcode sequencing for expt_115-NHP-serum-1000-escape   
3  SARS-CoV-2 RBD mutational antig Illumina barcode sequencing for expt_115-NHP-serum-1000-escape   

  library_strategy library_source library_selection library_layout  platform  \
0         AMPLICON      SYNTHETIC               PCR         single  ILLUMINA   
1         AMPLICON      SYNTHETIC               PCR         single  ILLUMINA   
2         AMPLICON      SYNTHETIC               PCR         single  ILLUMINA   
3         AMPLICON      SYNTHETIC               PCR         single  ILLUMINA   

      instrument_model                 design_description filetype  \
0  Illumina HiSeq 2500  PCR of barcodes from RBD variants    fastq   
1  Illumina HiSeq 2500  PCR of barcodes from RBD variants    fastq   
2  Illumina HiSeq 2500  PCR of barcodes from RBD variants    fastq   
3  Illumina HiSeq 2500  PCR of barcodes from RBD variants    fastq   

                                   filename  
0  expt_97-106_lib1_ref_S21_R1_001.fastq.gz  
1  expt_97-106_lib2_ref_S22_R1_001.fastq.gz  
2   expt_115_lib1_abneg_S17_R1_001.fastq.gz  
3   expt_115_lib2_abneg_S18_R1_001.fastq.gz

Now write the wide submissions data frame to a `*.tsv` file that can be used for uploading the SRA submission website:

In [8]:
submissions_spreadsheet = 'SRA_submission_spreadsheet.tsv'

submissions_wide.to_csv(submissions_spreadsheet, sep='\t', index=False)

We also want to write a file with all of the FASTQ files that are in the submission spreadsheet:

In [9]:
fastq_file_list = 'FASTQs_to_upload.csv'

print(f"Writing list of all the FASTQ files to upload to {fastq_file_list}")

(submissions_tidy
 [['filename_fullpath', 'filename']]
 .to_csv(fastq_file_list, index=False)
 )

Writing list of all the FASTQ files to upload to FASTQs_to_upload.csv
